# Lab CMBLenses

## Model

In [1]:
using CMBLensing, CMBLensingInferenceTestProblem
using CUDA, LaTeXStrings, MCMCDiagnosticTools, Plots, ProgressMeter, Random, Zygote
using LinearAlgebra, Statistics
ENV["LINES"] = 10;

LoadError: ArgumentError: Package CMBLensing [b60c06c0-7e54-11e8-3788-4bd722d65317] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [2]:
using Revise
using MicroCanonicalHMC

[ Info: Precompiling Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
[ Info: Precompiling MicroCanonicalHMC [234d2aa0-2291-45f7-9047-6fa6f316b0a8]
ERROR: LoadError: ArgumentError: Package CMBLensing [b60c06c0-7e54-11e8-3788-4bd722d65317] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

Stacktrace:
 [1] _require(pkg::Base.PkgId, env::String)
   @ Base ./loading.jl:1733
 [2] _require_prelocked(uuidkey::Base.PkgId, env::String)
   @ Base ./loading.jl:1619
 [3] macro expansion
   @ ./loading.jl:1607 [inlined]
 [4] macro expansion
   @ ./lock.jl:267 [inlined]
 [5] require(into::Module, mod::Symbol)
   @ Base ./loading.jl:1570
 [6] include
   @ ./Base.jl:457 [inlined]
 [7] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::Nothing)
   @ Base ./loading.jl:2004
 [8] top-level scope
  

LoadError: Failed to precompile MicroCanonicalHMC [234d2aa0-2291-45f7-9047-6fa6f316b0a8] to "/home/jaimerz/.julia/compiled/v1.9/MicroCanonicalHMC/jl_Dvnrmk".

In [3]:
prob = load_cmb_lensing_problem(storage = Array,
                                T = Float32,
                                Nside = 64);

LoadError: UndefVarError: `load_cmb_lensing_problem` not defined

In [4]:
target = CMBLensingTarget(prob);

LoadError: UndefVarError: `CMBLensingTarget` not defined

In [5]:
d = length(prob.Ωstart[:])
to_vec, from_vec = CMBLensingInferenceTestProblem.to_from_vec(prob.Ωstart);

LoadError: UndefVarError: `prob` not defined

## Sample

In [6]:
#spl=MCHMC(10.0, 10.0*sqrt(50); sigma=one(LenseBasis(diag(target.Λmass))))
spl = MCHMC(500, 0.0001;  adaptive=true, init_eps=10.0, init_L=10.0*sqrt(50), sigma=one(LenseBasis(diag(target.Λmass))))

LoadError: UndefVarError: `target` not defined

In [7]:
samples = Sample(spl, target, 1000, dialog=false, progress=true)

LoadError: UndefVarError: `Sample` not defined

In [8]:
samples_redux = []
for sample in samples
    if all(isfinite.(sample))
            append!(samples_redux, [sample])
    end
end

LoadError: UndefVarError: `samples` not defined

In [9]:
_samples = zeros(length(samples), length(samples[1]), 1)
_samples[:, :, 1] = mapreduce(permutedims, vcat, samples)
_samples = permutedims(_samples, (1,3,2));

LoadError: UndefVarError: `samples` not defined

In [10]:
ess, rhat = MCMCDiagnosticTools.ess_rhat(_samples[:,:,end÷2-1:end÷2])

LoadError: UndefVarError: `MCMCDiagnosticTools` not defined

In [11]:
_ess = minimum(ess)

LoadError: UndefVarError: `ess` not defined

In [12]:
_rhat = maximum(rhat)

LoadError: UndefVarError: `rhat` not defined

## Plotting

In [13]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

LoadError: UndefVarError: `Plots` not defined

In [14]:
rs = [sample[1:d][end-1] for sample in samples_redux]
Aϕs = [sample[1:d][end] for sample in samples_redux];

In [15]:
p_rs = [sample[d:2d][end-1] for sample in samples_redux]
p_Aϕs = [sample[d:2d][end] for sample in samples_redux];

In [16]:
plot(exp.(rs), label=L"r", xlabel="step")
plot!(exp.(Aϕs), label=L"A_\phi")


LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[16]:1

In [17]:
plot(exp.(p_rs), label=L"\Pi(r)", xlabel="step")
plot!(exp.(p_Aϕs), label=L"\Pi(A_\phi)")


LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[17]:1

In [18]:
Energy = [sample[end-1] for sample in samples_redux];
std(Energy)^2/d

LoadError: UndefVarError: `std` not defined

In [19]:
plot(Energy/d, label=L"Energy/d", xlabel="step")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[19]:1

In [20]:
plot(histogram(exp.(rs), xlabel=L"r", label=nothing, lw=1),
    histogram(exp.(Aϕs), xlabel=L"A_\phi", label=nothing, lw=1))

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[20]:1

In [21]:
to_vec, from_vec = CMBLensingInferenceTestProblem.to_from_vec(prob.Ωstart)
last_sample = from_vec(samples[end][1:end-2]);

LoadError: UndefVarError: `CMBLensingInferenceTestProblem` not defined

In [22]:
ps = map([(:ϕ°,:I,L"L",L"\phi^\circ"), (:f°,:E,"L\ell",L"E^\circ"), (:f°,:B,L"\ell",L"B^\circ")]) do (k1, k2, xlabel, title)
    plot(get_Cℓ(prob.Ωtrue[k1][k2]); label="true", xlabel, title)
    plot!(get_Cℓ(prob.Ωstart[k1][k2]); label="start", xlabel, title)
    plot!(get_Cℓ(last_sample[k1][k2]); label="last sample", xlabel, title)
end
plot(ps..., layout=(1,3), xscale=:log10, yscale=:log10, size=(1000,300), legend=:bottomleft)

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[22]:1

In [23]:
samps = [∇²*from_vec(sample[1:end-2]).ϕ°/2 for sample in samples[100:end]]
anim = @animate for i in 1:length(samps)
    motionblur = [0.5, 1, 0.5]
    frame = sum(circshift(samps, i)[1:length(motionblur)] .* motionblur) / sum(motionblur)
    plot(1 * frame, clims=(-0.5, 0.5), c=:thermal, title="κ samples")
end;

LoadError: UndefVarError: `samples` not defined

In [24]:
mp4(anim, "kappa_samples.mp4", fps = 25)

LoadError: UndefVarError: `mp4` not defined